## Table of Contents

1. [Setup](#setup)
2. [Responses API Basics](#responses-api)
3. [Reasoning Effort Parameter](#reasoning-effort)
4. [Verbosity Control](#verbosity)
5. [Tool Use with Reasoning](#tools)
6. [Structured Outputs](#structured)
7. [Best Practices](#best-practices)
8. [Practical Examples](#examples)
9. [Summary](#summary)

In [1]:
import os
import getpass
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file (if it exists)
load_dotenv()

# Set API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

client = OpenAI()
print("OpenAI client initialized (ready for API calls)")

OpenAI client initialized (ready for API calls)


In [3]:
client.responses.create(
    model="gpt-5.2",
    input="What is the capital of France?"
)

Response(id='resp_070c1b6432533ebc006984cdc915548194bb37cb2ef11b1ae6', created_at=1770311113.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5.2-2025-12-11', object='response', output=[ResponseOutputMessage(id='msg_070c1b6432533ebc006984cdc997bc819497203436ae06f9f2', content=[ResponseOutputText(annotations=[], text='Paris.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=0.98, background=False, completed_at=1770311113.0, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, prompt_cache_retention=None, reasoning=Reasoning(effort='none', generate_summary=None, summary=None), safety_identifier=None, service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='disable

In [4]:
response = client.responses.create(
    model="gpt-5.2",
    input="What is the capital of France?"
)

response.output_text

'Paris.'

In [8]:
prompt = """
Think through deeply and write a complex 3 paragraph story with 5 plot twists.
""" 

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "medium"},
    input=[{"role": "user", "content": prompt}]
)

response.output_text

'The rain in Pelagic District fell upward, pulled by the city’s old desalination towers like a prayer being unsaid, and Mara Kade walked beneath it with her collar up and her badge turned inward so the cameras wouldn’t catch the brass. She had been hired to solve a murder that the Council insisted must remain “administratively dead”: the body of Councilor Venn lay in a museum vault among relics that had never existed, his throat cut with a blade too clean to belong to any street gang. Mara’s only lead was a journal found in Venn’s coat—salt-stiff pages, meticulous handwriting—and she expected politics, blackmail, the usual rot. Instead, the first line made her stop cold in the humming dark: *Mara, you will believe this is the first time.* The next pages mapped her footsteps through the vault, predicted the questions she would ask, even recorded her habit of tapping her ring against her tooth when she lied to herself. It was her own handwriting, down to a childhood flourish she’d never 

In [ ]:
# reasoning tokens? - https://platform.openai.com/tokenizer
print(f"Reasoning tokens: {response.usage.output_tokens_details.reasoning_tokens}")

Reasoning tokens: 419


What do the intermediate reasoning outputs look like? 

In [19]:
response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "medium",
               "summary": "auto"},
    input=[{"role": "user", "content": prompt}]
)

response

Response(id='resp_0d87866dedd2495d006984d028d70c8193ac9559f28c815033', created_at=1770311720.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5.2-2025-12-11', object='response', output=[ResponseReasoningItem(id='rs_0d87866dedd2495d006984d0292e608193a762c0dc3b9f8cc9', summary=[Summary(text='**Crafting a complex narrative**\n\nI need to create a compelling story in three paragraphs, packed with five plot twists. I should show complexity through misdirection, so the twists aren’t obvious at first. The first twist will be revealing that the narrator is actually an AI. Then, I’ll make the detective the target, followed by the twist where the missing person is the narrator. The final two twists will reveal that the world is a simulation, and the narrator is either the villain or an imaginary character. I’ll outline the plot with main character Mara discovering a diary connected to an architect.', type='summary_text'), Summary(text="**Structuring a complex my

In [23]:
response.output[0].summary

[Summary(text='**Crafting a complex narrative**\n\nI need to create a compelling story in three paragraphs, packed with five plot twists. I should show complexity through misdirection, so the twists aren’t obvious at first. The first twist will be revealing that the narrator is actually an AI. Then, I’ll make the detective the target, followed by the twist where the missing person is the narrator. The final two twists will reveal that the world is a simulation, and the narrator is either the villain or an imaginary character. I’ll outline the plot with main character Mara discovering a diary connected to an architect.', type='summary_text'),
 Summary(text="**Structuring a complex mystery**\n\nI'm planning a noir mystery involving a missing cartographer named Dorian in a glass city beneath the ocean. The main character, Lio, is a courier who carries a sealed letter. The first twist will reveal that the letter is addressed to him from himself, dated tomorrow.\n\nIn the next paragraph, Li

In [30]:
question = "Write a 3 paragraph story with 5 plot twists"

response = client.responses.create(
    model="gpt-5.2",
    reasoning={"effort": "medium",
               "summary": "auto"},
    input=[{"role": "user", "content": question}],
    text={"verbosity": "high"} # controlling for the amount of text in the output
)

response

Response(id='resp_0448c6b5b18891a5006984d9747c6c81969844643aad47d461', created_at=1770314100.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5.2-2025-12-11', object='response', output=[ResponseReasoningItem(id='rs_0448c6b5b18891a5006984d974eec4819691acf7e2b961313c', summary=[Summary(text='**Crafting a story with twists**\n\nI\'m working on a three-paragraph story that includes five plot twists. The setup introduces Mara, a librarian who finds a mysterious map leading to a buried time capsule at a lighthouse, accompanied by her friend Jonah. \n\nIn the first twist, Jonah admits he was the one who sent the map to lure Mara. The second twist reveals the lighthouse has a secret facility underneath. In the second paragraph, they discover that the capsule contains Mara\'s future diary, suggesting she\'s in a time loop experiment. Jonah is her handler, and just as things intensify, the alarm sounds, and the "Director" appears—who is actually an older version

In [32]:
response.output[0].summary

[Summary(text='**Crafting a story with twists**\n\nI\'m working on a three-paragraph story that includes five plot twists. The setup introduces Mara, a librarian who finds a mysterious map leading to a buried time capsule at a lighthouse, accompanied by her friend Jonah. \n\nIn the first twist, Jonah admits he was the one who sent the map to lure Mara. The second twist reveals the lighthouse has a secret facility underneath. In the second paragraph, they discover that the capsule contains Mara\'s future diary, suggesting she\'s in a time loop experiment. Jonah is her handler, and just as things intensify, the alarm sounds, and the "Director" appears—who is actually an older version of Mara from the future. \n\nThe final twist comes when older Mara offers to let Jonah go but only if it means erasing him from existence, and the story concludes with a revelation that the reader is actually Mara\'s diary being written in real time!', type='summary_text'),
 Summary(text="**Refining the stor

Full raw intermediate reasoning output is not available. 